In [ ]:
import os
import numpy as np
import xarray as xr
import zarr
import s3fs
from matplotlib import pyplot as plt

# ---> set regen_zarr to True for first run
regen_zarr = False

# ----> create a fake data
ncfn0  = 'small.nc'
outdir = os.getcwd()
ncfn   = os.path.join(outdir,ncfn0)

x = np.linspace(-3,3,61)
y = np.linspace(-2,2,41)
t = np.linspace(-1,1,21)
xx,yy = np.meshgrid(x,y)
nt = len(t)
ny,nx = xx.shape
z = np.zeros((nt,ny,nx))
for i in range(nt):
    z[i] = xx**2*np.cos(t[i]) - yy**2*np.sin(t[i])
da = xr.DataArray(z,coords=[t,y,x],dims=['t','y','x'])
ds = xr.Dataset({'z':da})
ds.to_netcdf(ncfn)


# ----> convert nc to local zarr:: 3 chunks in t
zarr_local = os.path.join(ncfn+'.zarr')
if not os.path.exists(zarr_local): 
    # without the condition above, running this section 
    # a second time will produce error
    with xr.open_dataset(ncfn,chunks={'t':7}) as ds0:
        compressor = zarr.Blosc(cname='zstd', clevel=3)
        encoding = {vname: {'compressor': compressor} for vname in ds0.data_vars}
        ds.to_zarr(store=zarr_local,encoding=encoding) 


# ----> convert nc to zarr on s3:: 3 chunks in t, 
SNAME    = 'test_zarr'
S3_root  = 's3://pangeo-data-upload-oregon/icesat2/HMA_Validation/'
ZAR_path = os.path.join( S3_root , SNAME )
zpath    = os.path.join(ZAR_path,ncfn0+'.zarr')

fs       = s3fs.S3FileSystem()
ds_store = s3fs.S3Map(root=zpath,s3=fs,check=True)
# ---> need to remove remote zarr before regen_zarr !!
#      aws s3 rm s3://pangeo-data-upload-oregon/icesat2/HMA_Validation/test_zarr/small.nc.zarr/ --recursive
if regen_zarr: 
    with xr.open_dataset(ncfn,chunks={'t':7}) as ds1:
        compressor = zarr.Blosc(cname='zstd', clevel=3)
        encoding   = {vname: {'compressor': compressor} for vname in ds1.data_vars}
        ds.to_zarr(store=ds_store,encoding=encoding) 



In [ ]:
# --- Pangeo does not have ncview, check data here
it = 15
plt.contourf(x,y,z[it])

In [ ]:
# ----> check local zarr data
zarr_ds = xr.open_zarr(store=zarr_local)
print(zarr_ds)
plt.contourf(zarr_ds.x,zarr_ds.y,zarr_ds.z[it])

In [ ]:
# ---> check s3 zarr
dstore = s3fs.S3Map(root=zpath,s3=fs,check=True)
# --- method 1: empty, zarr_s3['z'] returns error
zarr_s3 = xr.open_zarr(store=dstore)
# --- method 2: looks empty, can access z by zarr_rt['t'], but no metadata
zarr_rt = zarr.group(store=dstore)
plt.contourf(zarr_rt.x,zarr_rt.y,zarr_rt.z[it])